In [1]:
import pandas as pd

# Load data

In [41]:
all_jobs_df = pd.read_csv(
    'https://lehd.ces.census.gov/data/lodes/LODES7/ca/wac/ca_wac_S000_JT00_2017.csv.gz',
    dtype={'w_geocode': str},
)
all_workers_df = pd.read_csv(
    'https://lehd.ces.census.gov/data/lodes/LODES7/ca/rac/ca_rac_S000_JT00_2017.csv.gz',
    dtype={'h_geocode': str},
)
cali_crosswalk_df = pd.read_csv(
    'https://lehd.ces.census.gov/data/lodes/LODES7/ca/ca_xwalk.csv.gz',
    dtype={'tabblk2010': str},
)

/Users/sidharthkapur/.local/share/virtualenvs/sidharthkapur-74SaHedF/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (28,29,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
housing_df = pd.read_excel(
    'http://www.dof.ca.gov/Forecasting/Demographics/Estimates/E-5/documents/E-5_2019_Internet%20Version.xlsx',
    sheet_name='E5CityCounty2019', header=3, skipfooter=7, usecols=range(13)
)
housing_df['County / City'] = housing_df['County / City'].str.strip()
# housing_df = housing_df[
#     (housing_df['County / City'] != 'County Total')
#     & (housing_df['County / City'] != 'Incorporated')
#     & (housing_df['County / City'] != 'Balance Of County') # todo, we could extract the county for this row if we wanted
#     & housing_df['Total'].notnull()
# ].copy()
# housing_df.rename(columns={'County / City': 'City'}, inplace=True)

In [4]:
county_rows_raw = housing_df[
    (
        housing_df['County / City'].str.contains('County').fillna(False)
        & (housing_df['County / City'] != 'Balance Of County')
    )
    | (housing_df['County / City'] == 'San Francisco')
]
rows_iter = county_rows_raw.iterrows()
county_rows = []
while True:
    try:
        row_1 = next(rows_iter)[1]
    except StopIteration:
        break
    
    row_2 = next(rows_iter)[1]
    assert row_2['County / City'] in ['County Total', 'San Francisco']
    row_1_values = row_1.drop(['County / City'])
    assert (row_1_values.isnull() | (row_1_values == ' ')).all()
    fixed_row = row_2.copy()
    fixed_row['County / City'] = row_1['County / City']
    county_rows.append(fixed_row)
counties_df = pd.DataFrame(county_rows)

In [60]:
jobs_by_block_df = all_jobs_df[['w_geocode', 'C000']].rename(columns={'C000': 'Jobs'}).merge(
    all_workers_df[['h_geocode', 'C000']].rename(columns={'C000': 'Workers'}),
    left_on='w_geocode',
    right_on='h_geocode',
    how='outer'
)

# We did an outer join, so some rows will have one or the other as null
jobs_by_block_df['geocode'] = jobs_by_block_df['w_geocode'].fillna(jobs_by_block_df['h_geocode'])

jobs_by_block_df = jobs_by_block_df.merge(
    cali_crosswalk_df[['tabblk2010', 'ctyname', 'trctname', 'cbsaname', 'stplcname']],
    left_on='w_geocode',
    right_on='tabblk2010',
    how='left'
)

# Look at jobs and housing data

In [87]:
jobs_by_block_df.groupby('cbsaname')['Jobs'].sum().sort_values(ascending=False).apply('{:,.0f}'.format)[:10]

cbsaname
Los Angeles-Long Beach-Anaheim, CA         6,249,699
San Francisco-Oakland-Hayward, CA          2,400,290
Riverside-San Bernardino-Ontario, CA       1,439,654
San Diego-Carlsbad, CA                     1,403,191
San Jose-Sunnyvale-Santa Clara, CA         1,077,279
Sacramento--Roseville--Arden-Arcade, CA      951,760
Fresno, CA                                   370,040
Oxnard-Thousand Oaks-Ventura, CA             310,992
Bakersfield, CA                              284,739
Stockton-Lodi, CA                            245,891
Name: Jobs, dtype: object

In [62]:
bay_area_counties = ['Alameda', 'Contra Costa', 'Marin', 'San Francisco', 'San Mateo', 'Santa Clara', 'Napa', 'Sonoma', 'Solano']
bay_area_jobs = jobs_by_block_df[
    jobs_by_block_df['ctyname'].isin([c + ' County, CA' for c in bay_area_counties])
].groupby('ctyname')['Jobs'].sum().sort_values(ascending=False)

In [88]:
bay_area_jobs.apply('{:,.0f}'.format)

ctyname
Santa Clara County, CA      1,060,260
Alameda County, CA            788,852
San Francisco County, CA      723,907
San Mateo County, CA          399,024
Contra Costa County, CA       375,252
Sonoma County, CA             201,244
Solano County, CA             148,424
Marin County, CA              113,255
Napa County, CA                74,858
Name: Jobs, dtype: object

In [89]:
bay_area_jobs.sum()

3885076.0

In [90]:
2400 + 1077

3477

In [91]:
3885076.0 / 0.97

4005232.989690722

In [67]:
bay_area_workers = jobs_by_block_df[
    jobs_by_block_df['ctyname'].isin([c + ' County, CA' for c in bay_area_counties])
].groupby('ctyname')['Workers'].sum().sort_values(ascending=False)

In [68]:
bay_area_workers.apply('{:,.0f}'.format)

ctyname
Santa Clara County, CA      767,721
Alameda County, CA          644,906
San Francisco County, CA    449,056
Contra Costa County, CA     377,530
San Mateo County, CA        308,690
Sonoma County, CA           174,861
Solano County, CA           136,485
Marin County, CA             91,105
Napa County, CA              51,671
Name: Workers, dtype: object

In [69]:
bay_area_workers.sum()

3002025.0

In [70]:
bay_area_housing_df = counties_df[
    counties_df['County / City'].isin([c + ' County' for c in bay_area_counties])
]

In [71]:
bay_area_housing_df

,County / City,Total,Household,Group Quarters,Total.1,Single Detached,Single Attached,Two to Four,Five Plus,Mobile Homes,Occupied,Vacancy Rate,Persons per Household
19,Alameda County,1669301.0,1629480.0,39821.0,605977.0,318177.0,47620.0,66235.0,166086.0,7859.0,572918.0,0.054555,2.844177
95,Contra Costa County,1155879.0,1145404.0,10475.0,416062.0,277588.0,32009.0,28626.0,70556.0,7283.0,389644.0,0.063495,2.939617
361,Marin County,262879.0,254596.0,8283.0,112394.0,68750.0,11354.0,8308.0,21990.0,1992.0,104289.0,0.072112,2.441255
442,Napa County,140779.0,135918.0,4861.0,55180.0,37402.0,3016.0,4104.0,6857.0,3801.0,49581.0,0.101468,2.741332
641,San Francisco County,883869.0,858066.0,25803.0,399372.0,65860.0,59605.0,81520.0,191785.0,602.0,365197.0,0.085572,2.350000
699,San Mateo County,774485.0,765077.0,9408.0,279248.0,156554.0,25516.0,17704.0,76301.0,3173.0,266217.0,0.046665,2.873885
738,Santa Clara County,1954286.0,1924072.0,30214.0,671439.0,350878.0,65159.0,49226.0,187252.0,18924.0,641019.0,0.045306,3.001583
802,Solano County,441307.0,429845.0,11462.0,159586.0,114555.0,6798.0,11274.0,22346.0,4613.0,148301.0,0.070714,2.898463
819,Sonoma County,500675.0,490526.0,10149.0,204976.0,138077.0,14675.0,13852.0,26919.0,11453.0,186384.0,0.090703,2.631803


In [72]:
bay_area_housing_df['Total.1'].sum()

2904234.0

In [73]:
bay_area_housing_df['Occupied'].sum()

2723550.0

In [74]:
3885076.0 / 2904234.0

1.3377282959981875

In [75]:
3885076.0 / 2723550.0

1.426475005048558

In [95]:
5510377 / 2904234

1.897359854612266

In [96]:
5510377 / 2723550

2.023233280093995

In [97]:
(1.897 - 1.5) * 2904234

1152980.898